<a href="https://colab.research.google.com/github/roncamposj/Computer-Vision-Introductory-Assignments/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 1

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

#torch.ones is an input and torchrand(requires_grad) = a parameter, you update it.

#loss = -y*torch.log(p)-(1-y)*torch.log(1-p)

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        
        # Define various layers here
        self.flatten = nn.Flatten()
        #for model 1 and 2 use
        self.fc1 = nn.Linear(28*28, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 10)
        
        #for model 3 use
        self.alt1 = nn.Linear(28*28, 202)
        self.alt2 = nn.Linear(202, 202)
        self.alt3 = nn.Linear(202, 202)
        self.alt4 = nn.Linear(202, 202)
        self.alt5 = nn.Linear(202, 10)
        
        # This will select the forward pass function based on mode for the ConvNet.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
      
    # Baseline sample model
    def model_0(self, X):
        # ======================================================================
        # Three fully connected layers with activation
        
        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = torch.sigmoid(X)
                
        return X  
        
    # Baseline model. task 1
    def model_1(self, X):
        # ======================================================================
        # Three fully connected layers without activation


        X = self.flatten(X)
        # NotImplementedError
        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc3(X)
                        
        return X
        

    # task 2
    def model_2(self, X):
        # ======================================================================
        # Train with activation (use model 1 from task 1)

        X = self.flatten(X)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = torch.sigmoid(X)
        
        return X

	
    # task 3
    def model_3(self, X):
        # ======================================================================
        # Change number of fully connected layers and number of neurons from model 2 in task 2

        X = self.flatten(X)
        X = self.alt1(X)
        X = F.relu(X)
        X = self.alt2(X)
        X = F.relu(X)
        X = self.alt3(X)
        X = F.relu(X)
        X = self.alt4(X)
        X = F.relu(X)
        X = self.alt5(X)
        X = torch.sigmoid(X)

        
        return X       
        


In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
# from ConvNet import ConvNet 
import argparse
import numpy as np 


def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        
        # ======================================================================
        # Compute loss based on criterion
        # ----------------- YOUR CODE HERE ----------------------
        #
        # Remove NotImplementedError and assign correct loss function.

        loss = criterion(output,target)
        #F.nll_loss(output, target)

        #nn.MSELoss() is an alternative for above.
        
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # ======================================================================
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc
    


def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            

            # Predict for data by doing forward pass
            output = model(data)
            
            # ======================================================================
            # Compute loss based on same criterion as training
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output,target)
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # ======================================================================
            # Count correct predictions overall 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy
    

def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    
    # Set proper device based on cuda availability 
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)
    
    # Initialize the model and send to device 
    model = ConvNet(FLAGS.mode).to(device)
    
    # Initialize the criterion for loss computation 
    # ======================================================================
    # Remove NotImplementedError and assign correct loss function.
    criterion = nn.CrossEntropyLoss()

    
    
    # Initialize optimizer type 
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)
    
    # Create transformations to apply to each data sample 
    # Can specify variations such as image flip, color flip, random crop, ...
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    
    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.MNIST('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size, 
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size, 
                                shuffle=False, num_workers=4)
    
    best_accuracy = 0.0
    
    # Run training for n_epochs specified in config 
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            
    print("accuracy is {:2.2f}".format(best_accuracy))
    
    print("Training and evaluation finished")
    
    
if __name__ == '__main__':
    # Set parameters for Sparse Autoencoder
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--mode',
                        type=int, default=1,
                        help='Select mode between 1-3.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.1,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=20,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=10,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str,
                        default='logs',
                        help='Directory to put logging.')
                        
    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()
    
    print("Mode: ", FLAGS.mode)
    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)
    
    run_main(FLAGS)
    

Mode:  3
LR:  0.1
Batch size:  10
Torch device selected:  cpu

Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train set: Average loss: 1.6040, Accuracy: 48601/60000 (81%)
Test set: Average loss: 1.5119, Accuracy: 9372/10000 (94%)

Epoch:  2
Train set: Average loss: 1.5019, Accuracy: 56960/60000 (95%)
Test set: Average loss: 1.4972, Accuracy: 9570/10000 (96%)

Epoch:  3
Train set: Average loss: 1.4906, Accuracy: 57762/60000 (96%)
Test set: Average loss: 1.5011, Accuracy: 9461/10000 (95%)

Epoch:  4
Train set: Average loss: 1.4851, Accuracy: 58150/60000 (97%)
Test set: Average loss: 1.4877, Accuracy: 9655/10000 (97%)

Epoch:  5
Train set: Average loss: 1.4825, Accuracy: 58396/60000 (97%)
Test set: Average loss: 1.4817, Accuracy: 9724/10000 (97%)

Epoch:  6
Train set: Average loss: 1.4805, Accuracy: 58492/60000 (97%)
Test set: Average loss: 1.4825, Accuracy: 9717/10000 (97%)

Epoch:  7
Train set: Average loss: 1.4788, Accuracy: 58655/60000 (98%)
Test set: Average loss: 1.4835, Accuracy: 9720/10000 (97%)

Epoch:  8
Train set: Average loss: 1.4777, Accuracy: 58732/60000 (98%)
Test set: Average loss

**Task 1:**

The final accuracy was 78.93, which can only be attributed to epoch 1.  The final training and testing accuracies at epoch 20 were abysmal. The training and testing accuracies started off promising - they were 76% and 79% respectively at epoch 1, but quickly took a rough turn. Both train and test accuracies stayed at exactly 10% for the rest of the entire model, and their average losses were NaN after that epoch.  

I attribute this bizarre behavior to the fact that there were no activation functions were implemented in any parts of the model.

This model took 11m 3s to execute.
_______________________________________________________________________________
**Task 2:**

This model gave me a final test accuracy of 95.57% and a final training accuracy of 96% at epoch 20.  Both the training and test accuracy reached their peaks at around epoch 2 and stagnated until epoch 20.  The average losses of the two did change a bit from the starting epoch to the final one (a difference of around 1 for the train set and of 0.4 for the test set)


This model took 11m 2s to execute.
_______________________________________________________________________________
**Task 3:**

With a final testing accuracy of 97.65%, this was by far my most successful model.  Accuracy for both the training and testing sets reach the high 90's, percentage wise early on (at around the fourth epoch).  The training set finished with an accuracy of 99% at epoch 20.  The average losses for both the training and testing surprisingly did not change too much from the first epoch to the last.

This model took 14m 10s to execute.

Something interesting to note, is that I tried the ( 2/3* input) + output advice that is suggested by some --vwhen choosing the layers input and neurons. This resulted in slightly better accuracy (around 98.2% accuracy), but a performance of 24m.  A whole 10m more than the model I ended up choosing (for this reason alone).

Part 2

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        
        # Define various layers here, such as in the tutorial example
        #model 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)

        #models 2 and 3
        self.m_conv1 = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3)
        self.m_conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)
        self.m_conv3 = nn.Conv2d(in_channels=40, out_channels=40, kernel_size=3)

        
        self.fc1_model1 = nn.Linear(360, 100)  # This is first fully connected layer for step 1.
        self.fc1_model2 = nn.Linear(1440, 100) # This is first fully connected layer for step 2.
        self.fc1_model3 = nn.Linear(640, 100)  # This is first fully connected layer for step 3
        
        self.fc2 = nn.Linear(100, 10)       # This is 2nd fully connected layer for all models.
        
        self.fc_model0 = nn.Linear(2250, 100)   # This is for example model.
        
        
        # This will select the forward pass function based on mode for the ConvNet.
        # Based on the question, you have 3 modes available for step 1 to 3.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        elif mode == 0:
            self.forward = self.model_0
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
        
    
    # Example model. Modify this for step 1-3
    def model_0(self, X):
        # ======================================================================         
        
        X = F.relu(self.conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)
        
        X = torch.flatten(X, start_dim=1)
        #print(X.shape)
        
        X = F.relu(self.fc_model0(X))
        X = self.fc2(X)
        
        return X
        
    
    # Simple CNN. step 1
    def model_1(self, X):
        # ======================================================================
         
        # Complete this part as model_0, add one more conv2d layer 
        # with relu activation followed by maxpool layer.

        X = F.relu(self.conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)
        X = F.relu(self.conv2(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        
        X = torch.flatten(X, start_dim=1)
        #print(X.shape)
        
        X = F.relu(self.fc1_model1(X))
        X = self.fc2(X)
        
        return X
        

    # Increase filters. step 2
    def model_2(self, X):
        # ======================================================================
        
        # Complete this part as model_1. Modify in/out channels for conv2d layers.
        
        X = F.relu(self.m_conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)
        X = F.relu(self.m_conv2(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        
        X = torch.flatten(X, start_dim=1)
        #print(X.shape)
        
        X = F.relu(self.fc1_model2(X))
        X = self.fc2(X)
        
        return X
        

    # Large CNN. step 3
    def model_3(self, X):
        # ======================================================================
        
        # Complete this part as model_2, add one more conv2d layer 
        # with relu activation. Do not add maxpool after this new conv2d layer.
        
        X = F.relu(self.m_conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)
        X = F.relu(self.m_conv2(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        X = F.relu(self.m_conv3(X))
        
        X = torch.flatten(X, start_dim=1)
        #print(X.shape)
        
        X = F.relu(self.fc1_model3(X))
        X = self.fc2(X)
        
        return X



   


In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import argparse
import numpy as np 

def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        # print(data.shape)
        # print(target.shape)
        # exit()
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        
        # ======================================================================
        # Compute loss based on criterion
        loss = criterion(output,target)
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # ======================================================================
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc
    


def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            

            # Predict for data by doing forward pass
            output = model(data)
            
            # ======================================================================
            # Compute loss based on same criterion as training
            criterion = nn.CrossEntropyLoss(reduction='mean')
            loss = criterion(output,target)
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # ======================================================================
            # Count correct predictions overall 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy
    

def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    
    # Set proper device based on cuda availability 
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)
    
    # Initialize the model and send to device 
    model = ConvNet(FLAGS.mode).to(device)
    # print(model)
    # exit()

    # Initialize the criterion for loss computation 
    criterion = nn.CrossEntropyLoss(reduction='mean')
    
    # Initialize optimizer type 
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)
    
    # Create transformations to apply to each data sample 
    # Can specify variations such as image flip, color flip, random crop, ...
    #transform=transforms.Compose([
    #    transforms.ToTensor(),
    #    transforms.Normalize((0.1307,), (0.3081,))
    #    ])
    
    transform = transforms.Compose(
                    [transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
     
    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.CIFAR10('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.CIFAR10('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size, 
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size, 
                                shuffle=False, num_workers=4)
    
    best_accuracy = 0.0
    
    # Run training for n_epochs specified in config 
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            
    print("accuracy is {:2.2f}".format(best_accuracy))
    
    print("Training and evaluation finished")
    
    
if __name__ == '__main__':
    # Set parameters for Sparse Autoencoder
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--mode',
                        type=int, default=3,
                        help='Select mode between 1-3.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.1,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=10,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=100,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str,
                        default='logs',
                        help='Directory to put logging.')
                        
    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()
    
    print("Mode: ", FLAGS.mode)
    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)
    
    run_main(FLAGS)
    
    


Mode:  3
LR:  0.1
Batch size:  100
Torch device selected:  cpu
Files already downloaded and verified

Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train set: Average loss: 1.9504, Accuracy: 13921/50000 (28%)
Test set: Average loss: 1.6096, Accuracy: 4172/10000 (42%)

Epoch:  2
Train set: Average loss: 1.5252, Accuracy: 22131/50000 (44%)
Test set: Average loss: 1.4445, Accuracy: 4778/10000 (48%)

Epoch:  3
Train set: Average loss: 1.3432, Accuracy: 25942/50000 (52%)
Test set: Average loss: 1.2614, Accuracy: 5487/10000 (55%)

Epoch:  4
Train set: Average loss: 1.2000, Accuracy: 28805/50000 (58%)
Test set: Average loss: 1.1634, Accuracy: 5863/10000 (59%)

Epoch:  5
Train set: Average loss: 1.0897, Accuracy: 30834/50000 (62%)
Test set: Average loss: 1.0810, Accuracy: 6184/10000 (62%)

Epoch:  6
Train set: Average loss: 0.9941, Accuracy: 32544/50000 (65%)
Test set: Average loss: 1.0995, Accuracy: 6090/10000 (61%)

Epoch:  7
Train set: Average loss: 0.9273, Accuracy: 33764/50000 (68%)
Test set: Average loss: 0.9815, Accuracy: 6512/10000 (65%)

Epoch:  8
Train set: Average loss: 0.8516, Accuracy: 35100/50000 (70%)
Test set: Average loss

**Task 4:**

The final training accuracy was 69% epoch 10, while the test accuracy was 63.65%.  The average losses dropped for both sets, but it seems that the test's average loss did not improve too much, as it only dropped around 0.4 between epoch 1 and 10.  Meanwhile, the average loss difference for the training set from epoch 1 and 10 was around 1.

This model took 4m 37s to execute.
_______________________________________________________________________________
**Task 5:**

This model gave me a final test accuracy of 69.19% and a final training accuracy of 80% at epoch 10.  While the training accuracy kept improving as the model went along, the testing accuracy sort of remained stagnant after epoch 6/7.

The average losses for both the train and test sets did linearly improve though, as epochs passed.

This model took 8m 15s to execute.
_______________________________________________________________________________
**Task 6:**

The final testing acurracy here was 68.44%.  The training set finished with an accuracy of 74% at epoch 10.  The average losses for both the training and testing sets dropped as new epochs ran.  The only peculiar behavior I can report is the testing accuracy also stagnated at a certain point, though only at epoch 9.  

As a sidenote, I was surprised that this models accuracy was very similar to the previous model.  I assumed that with an extra convolution layer, the performance would only be better.

This model took 8m 48s to execute.